In [ ]:
import numpy as np
import os
from pymongo import MongoClient
import utils.const as const
import re
import demoji
import xlsxwriter
import xlrd
import langid

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']
langid.set_languages(['gu','en','hi'])

In [ ]:
db=const.grpId
collection=const.PostCollection
dataH='dataHindi.xlsx'
dataE='dataEnglish.xlsx'
dataG='dataGujarati.xlsx'
dataHT='dataHindi_T.xlsx'
dataET='dataEnglish_T.xlsx'
dataGT='dataGujarati_T.xlsx'
idH='idH.txt'
idE='idE.txt'
idG='idG.txt'

arrH=[]
arrG=[]
arrE=[]

mergeDataFile='mergeData.txt'
mergeId='mergeId.txt'

row_limit=20000

In [ ]:
myCollect=client[db][collection]

In [ ]:
fdH=open(idH,'w')
fdE=open(idE,'w')
fdG=open(idG,'w')

In [ ]:
def process(s):
    s=s.replace('\n',' ')  ## new line
    
    s = re.sub('https?:\S*', ' ', s, flags=re.MULTILINE) ## links
    
    s=re.sub('[#$]+',' ',s) ## characters that are not required
    
    s=demoji.replace_with_desc(s, "$$") ## emoji graphic
    s=s.replace('$$',' ') 
    
    s=re.sub('\s+', ' ',s) ## extra space
    
    return s;

In [ ]:
# post comment
c=0
for post in myCollect.find({}):
    if 'comments' in post:
        for comment in post['comments']:
            m=''
            if 'message' in comment:
                m=comment['message']
                
            m=process(m)
            
            lang = langid.classify(m)[0]
            
            if lang=='en':
                arrE.append(m)
                fdE.write(post['_id']+' '+comment['_id']+'\n')
            elif lang=='gu':
                arrG.append(m)
                fdG.write(post['_id']+' '+comment['_id']+'\n')
            else:
                arrH.append(m)
                fdH.write(post['_id']+' '+comment['_id']+'\n')
                
            c=c+1
            
print(c)

In [ ]:
print(len(arrE),len(arrH),len(arrG))

In [ ]:
fdE.close()
fdH.close()
fdG.close()

In [ ]:
def createExcel(lines,outputFile):
    l=len(lines)
    print(l)

    colNum=(int)(l/row_limit)
    if colNum*row_limit != l:
        colNum=colNum+1
        
    print('cols',colNum)

    workbook = xlsxwriter.Workbook(outputFile)
    worksheet = workbook.add_worksheet()

    rowCount=0

    for i in range(row_limit):
        if i*colNum>=l:
            break;

        st=''
        for j in range(colNum):
            if i*colNum + j < l:
                st+=lines[i*colNum + j]
                st+=' $ '
            else:
                break
        if len(st)==0:
            continue;
            
        worksheet.write(i,0,st)
        rowCount+=1

    print('rows in sheet '+str(rowCount))

    workbook.close()

In [ ]:
createExcel(arrE,dataE)
createExcel(arrG,dataG)
createExcel(arrH,dataH)

## now upload outputFile in google sheet bring back the translated ones

In [ ]:
def mergeData(fp1,fp2,idFile,inputFile,l):  
    fp=open(idFile,'r');
    
    colNum=(int)(l/row_limit)
    if colNum*row_limit != l:
        colNum=colNum+1

    workbook = xlrd.open_workbook(inputFile)
    sheet = workbook.sheet_by_index(0)

    print('rows in sheet '+str(sheet.nrows))

    for i in range(sheet.nrows):
        st=str(sheet.cell_value(i, 0))
        arr=st.split('$')

        if len(arr) < colNum:
            print(i,len(arr),colNum+1)

        while len(arr)<colNum+1: ### +1 because of second last line of above cell does not have st[:-2] but st
            arr.append('')

        for k in arr[0:colNum]:
            fp1.write(k.strip()+"\n")
            fp2.write(fp.readline())
    
    fp.close()

In [ ]:
fp1=open(mergeDataFile,'w')
fp2=open(mergeId,'w')
mergeData(fp1,fp2,idH,dataHT,len(arrH))
mergeData(fp1,fp2,idE,dataET,len(arrE))
mergeData(fp1,fp2,idG,dataGT,len(arrG))

fp1.close()
fp2.close()